<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6e896140722f07c5f95b53d88f7b3daa334de8747b9482c66f514778127a932a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-21 14:42:14
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 38.88 K (0.28%)
Current PnL: -20.04 L (-13.35%)
CY Booked + Current PnL: -8.14 L (-5.42%)
-------------------
Total profit:  2.22 L
Total loss:  -22.26 L
-------------------
Total Booked + Current PnL: 18.84 L (15.3%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.41%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.21 L (61.66%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.2%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1668.50,-14.30,75.0,X-LC,4.19,228584.0,24084.0,0.0,1.79,11.78,0.00,11.78,10.0,inf,1.63,21.78,X40N,ATH,PHARMA
83,VOLTAS,1530.00,4.24,66.0,X-MC,2.03,218550.0,26808.0,10949.0,0.84,13.98,5.01,19.69,99.0,2.45,1.56,21.73,XY25,NTT,AC
56,RELIANCE,1533.00,-8.78,72.0,X-LC,2.85,228025.0,17659.0,11128.0,-0.35,8.39,4.88,13.68,37.0,1.59,1.62,26.30,XY25,NTT,REFINERIES
76,TTKPRESTIG,770.00,95.94,49.0,M-SC,1.84,84430.0,-16347.0,16439.0,1.11,-16.22,19.47,0.09,245.0,-0.99,0.60,10.92,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-13.84,64.0,X-MC,2.92,148370.0,12237.0,18888.0,-0.30,8.99,12.73,22.86,91.0,0.65,1.06,22.74,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-287.15,39.0,H-SC,6.58,104040.0,-59565.0,180884.0,3.08,-36.41,173.86,74.15,139.0,-0.33,0.74,33.36,XR,NTT,CHEMICALS
58,ROUTE,2236.84,-54.63,27.0,H-SC,21.02,74427.0,-58639.0,151496.0,1.94,-44.07,203.55,69.78,118.0,-0.39,0.53,1.94,SR,ATH,IT
53,RAJOOENG,143.10,-40.48,49.0,H-SC,18.00,90300.0,-12200.0,52798.0,1.93,-11.90,58.47,39.61,114.0,-0.23,0.64,10.28,AR,ATH,MISC
62,SFL,1287.00,32.84,50.0,M-SC,7.37,175401.0,-86817.0,156651.0,1.83,-33.11,89.31,26.63,229.0,-0.55,1.25,9.06,XY24,NTT,MISC
18,CIPLA,1668.50,-14.30,75.0,X-LC,4.19,228584.0,24084.0,0.0,1.79,11.78,0.00,11.78,10.0,inf,1.63,21.78,X40N,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.00,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
25,GREENPANEL,537.00,230.10,54.0,M-SC,4.74,147376.0,-35702.0,114673.0,-1.61,-19.50,77.81,43.14,230.0,-0.31,1.05,38.63,XY24,NTT,MISC
15,CAMPUS,393.00,-23.28,55.0,M-SC,2.89,159889.0,-7886.0,63732.0,-1.14,-4.70,39.86,33.28,210.0,-0.12,1.14,23.93,XY24,NTT,FOOTWEAR
29,HCLTECH,1922.01,1.94,57.0,X-LC,8.04,232627.0,-9297.0,69137.0,-0.92,-3.84,29.72,24.73,8.0,-0.13,1.66,10.81,X40,ATH,IT
71,TATAELXSI,9161.00,-24.89,43.0,H-MC,7.62,101660.0,-30175.0,72402.0,-0.87,-22.89,71.22,32.03,98.0,-0.42,0.72,13.87,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.0,41.14,47.0,M-MC,10.59,223240.0,-1722.0,169372.0,-0.15,-0.77,75.87,74.53,192.0,-0.01,1.59,31.24,XY24,BTT,STEEL
85,WIPRO,420.0,-14.70,43.0,M-LC,6.17,149569.0,-1376.0,110831.0,0.00,-0.91,74.10,72.51,53.0,-0.01,1.07,5.06,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.51,65.0,H-SC,2.00,230697.0,-38970.0,75300.0,1.10,-14.45,32.64,13.47,138.0,-0.52,1.64,17.30,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,95.94,49.0,M-SC,1.84,84430.0,-16347.0,16439.0,1.11,-16.22,19.47,0.09,245.0,-0.99,0.60,10.92,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.63,43.0,H-SC,2.61,139403.0,-36500.0,78526.0,0.31,-20.75,56.33,23.89,149.0,-0.46,0.99,20.39,OX40N,NTT,CERAMICS
65,SIS,528.0,2075.13,53.0,H-SC,3.59,87784.0,-23248.0,46859.0,0.22,-20.94,53.38,21.26,156.0,-0.50,0.63,18.60,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.08,207240.0,4070.0,38070.0,0.56,2.00,18.37,20.74,55.0,0.11,1.48,7.66,XY25,NTT,FINANCE
31,HINDZINC,730.22,28.52,51.0,M-LC,10.37,209844.0,4768.0,107797.0,-0.82,2.32,51.37,54.89,52.0,0.04,1.50,26.98,X5K,ATH,METALS
45,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127184.0,1887.0,57042.0,1.63,1.51,44.85,47.03,141.0,0.03,0.91,59.25,MH,ATH,POWER
78,UNIONBANK,163.00,-11.63,58.0,M-LC,8.87,160522.0,19682.0,24817.0,-0.11,13.97,15.46,31.59,66.0,0.79,1.14,44.16,XY24,NTT,BANKS
11,BANKINDIA,190.00,-22.98,66.0,H-MC,13.02,200017.0,20209.0,92208.0,0.22,11.24,46.10,62.52,88.0,0.22,1.43,49.16,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-22.98,66.0,H-MC,13.02,200017.0,20209.0,92208.0,0.22,11.24,46.10,62.52,88.0,0.22,1.43,49.16,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-39.28,65.0,M-SC,4.12,102578.0,11080.0,67671.0,0.72,12.11,65.97,86.07,223.0,0.16,0.73,48.96,XR,NTT,DURABLES
31,HINDZINC,730.22,28.52,51.0,M-LC,10.37,209844.0,4768.0,107797.0,-0.82,2.32,51.37,54.89,52.0,0.04,1.50,26.98,X5K,ATH,METALS
45,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127184.0,1887.0,57042.0,1.63,1.51,44.85,47.03,141.0,0.03,0.91,59.25,MH,ATH,POWER
36,INDIAMART,4810.62,-54.83,45.0,H-SC,7.54,125917.0,2581.0,129040.0,1.76,2.09,102.48,106.72,119.0,0.02,0.90,25.96,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.49,145454.0,-131069.0,241134.0,0.24,-47.40,165.78,39.81,54.0,-0.54,1.04,2.52,XY24,NTT,AUTO
51,QUESS,986.00,-47.11,39.0,X-SC,38.74,52461.0,-12545.0,163474.0,0.76,-19.30,311.61,232.18,198.0,-0.08,0.37,2.33,XY24,NTT,MISC
28,HAVELLS,2069.16,-6.60,42.0,X-MC,2.63,231410.0,-16321.0,91384.0,-0.13,-6.59,39.49,30.30,92.0,-0.18,1.65,5.83,X40,ATH,ELECTRICAL
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,155250.0,-30845.0,78324.0,0.26,-16.57,50.45,25.51,15.0,-0.39,1.11,14.41,AR,ATH,ELECTRICAL
55,RELAXO,1176.00,-39.01,46.0,X-SC,4.83,83087.0,-62073.0,137999.0,1.76,-42.76,166.09,52.31,136.0,-0.45,0.59,11.22,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.47,43.0,H-LC,0.47,155250.0,-30845.0,78324.0,0.26,-16.57,50.45,25.51,15.0,-0.39,1.11,14.41,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,4.24,66.0,X-MC,2.03,218550.0,26808.0,10949.0,0.84,13.98,5.01,19.69,99.0,2.45,1.56,21.73,XY25,NTT,AC
41,ITC,452.00,-37.50,60.0,X-LC,2.53,201933.0,1795.0,19103.0,-0.02,0.90,9.46,10.44,4.0,0.09,1.44,6.51,X40,NTT,FMCG
28,HAVELLS,2069.16,-6.60,42.0,X-MC,2.63,231410.0,-16321.0,91384.0,-0.13,-6.59,39.49,30.30,92.0,-0.18,1.65,5.83,X40,ATH,ELECTRICAL
56,RELIANCE,1533.00,-8.78,72.0,X-LC,2.85,228025.0,17659.0,11128.0,-0.35,8.39,4.88,13.68,37.0,1.59,1.62,26.30,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,39.0,X-SC,38.74,52461.0,-12545.0,163474.0,0.76,-19.30,311.61,232.18,198.0,-0.08,0.37,2.33,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,15.0,X-LC,11.49,145454.0,-131069.0,241134.0,0.24,-47.40,165.78,39.81,54.0,-0.54,1.04,2.52,XY24,NTT,AUTO
2,ACC,3906.00,-37.64,50.0,X-MC,3.14,184940.0,-53011.0,205653.0,1.01,-22.28,111.20,64.15,174.0,-0.26,1.32,3.74,XY24,BTT,CEMENT
73,TCS,4389.97,-29.33,50.0,X-LC,12.69,282028.0,-63928.0,130635.0,-0.49,-18.48,46.32,19.28,1.0,-0.49,2.01,4.26,X40,ATH,IT
9,BAJAJHFL,181.50,-13.84,49.0,X-MC,5.58,181417.0,-19089.0,116252.0,1.44,-9.52,64.08,48.45,90.0,-0.16,1.29,4.62,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,39.0,X-SC,38.74,52461.0,-12545.0,163474.0,0.76,-19.30,311.61,232.18,198.0,-0.08,0.37,2.33,XY24,NTT,MISC
55,RELAXO,1176.00,-39.01,46.0,X-SC,4.83,83087.0,-62073.0,137999.0,1.76,-42.76,166.09,52.31,136.0,-0.45,0.59,11.22,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.29,47.0,X-SC,4.22,96778.0,-31892.0,77190.0,0.10,-24.79,79.76,35.20,219.0,-0.41,0.69,11.08,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.61,49.0,X-SC,0.90,107307.0,5303.0,38620.0,0.11,5.20,35.99,43.06,122.0,0.14,0.76,23.20,X40N,ATH,MISC
32,HONAUT,58357.33,-22.51,50.0,X-SC,2.54,109290.0,-18648.0,65782.0,0.23,-14.58,60.19,36.84,143.0,-0.28,0.78,11.89,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.33,50.0,X-LC,12.69,282028.0,-63928.0,130635.0,-0.49,-18.48,46.32,19.28,1.0,-0.49,2.01,4.26,X40,ATH,IT
39,INFY,2275.00,-19.60,48.0,X-LC,7.60,313323.0,435.0,171262.0,0.69,0.14,54.66,54.87,3.0,0.00,2.23,6.80,X40,BTT,IT
41,ITC,452.00,-37.50,60.0,X-LC,2.53,201933.0,1795.0,19103.0,-0.02,0.90,9.46,10.44,4.0,0.09,1.44,6.51,X40,NTT,FMCG
82,VBL,671.64,-18.69,52.0,X-LC,10.39,292938.0,-22904.0,135572.0,0.61,-7.25,46.28,35.67,5.0,-0.17,2.09,5.60,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,53.0,H-LC,6.70,251448.0,-10171.0,129395.0,0.13,-3.89,51.46,45.57,7.0,-0.08,1.79,6.99,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7188.89,70.0,L-SC,6.73,79989.0,-13821.0,90436.0,1.66,-14.73,113.06,81.67,269.0,-0.15,0.57,54.90,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,30.11,83021.0,-30528.0,70576.0,1.50,-26.89,85.01,35.27,268.0,-0.43,0.59,103.20,XR,NTT,HOTELS
48,MASFIN,398.61,-19.15,51.0,H-SC,9.47,92100.0,-5880.0,27483.0,1.15,-6.00,29.84,22.05,152.0,-0.21,0.66,33.87,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-39.28,65.0,M-SC,4.12,102578.0,11080.0,67671.0,0.72,12.11,65.97,86.07,223.0,0.16,0.73,48.96,XR,NTT,DURABLES
45,KPIGREEN,731.05,6.72,67.0,H-SC,5.54,127184.0,1887.0,57042.0,1.63,1.51,44.85,47.03,141.0,0.03,0.91,59.25,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-190.91,75.0,M-SC,21.09,157491.0,8271.0,135915.0,1.74,5.54,86.30,96.62,208.0,0.06,1.12,70.44,XY25,NTT,FINANCE
68,SURYODAY,240.0,68.86,64.0,H-SC,11.91,154510.0,-24561.0,90296.0,1.75,-13.72,58.44,36.71,135.0,-0.27,1.10,53.30,XR,NTT,BANKS
77,UJJIVANSFB,60.0,113.54,62.0,H-SC,13.60,135000.0,-7479.0,27000.0,-2.46,-5.25,20.00,13.70,163.0,-0.28,0.96,59.34,OX40N,NTT,BANKS
4,ANGELONE,3033.0,15.82,69.0,X-SC,7.69,204672.0,13666.0,44025.0,-0.14,7.15,21.51,30.21,157.0,0.31,1.46,29.53,X40N,NTT,FINANCE
57,REPCOHOME,880.0,-56.04,68.0,H-SC,2.24,251526.0,-34979.0,292324.0,0.07,-12.21,116.22,89.82,134.0,-0.12,1.79,33.08,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.06
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    33.76
MC    21.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.50
X40      14.75
XY25     12.41
XR       11.14
X40N     10.27
AR        8.00
OX40N     7.65
X200      1.78
X5K       1.50
SR        1.11
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.86
X-MC    16.01
M-SC    12.82
M-LC     5.19
H-LC     4.68
X-SC     4.65
H-MC     3.81
M-MC     1.59
L-SC     1.50
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.13,-3.56,37.69
IT,12.58,-19.12,83.42
FINANCE,12.25,-10.00,59.86
MISC,6.94,-17.82,83.11
BANKS,6.52,-9.01,66.35
PAINTS,5.69,-15.64,32.88
ELECTRICAL,5.33,-10.17,48.82
INSURANCE,3.93,-0.52,38.09
AUTO,2.82,-45.82,115.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3291811.0,22
XR,1348219.0,14
AR,1207120.0,9
X40,767793.0,10
X40N,587267.0,8
OX40N,572245.0,10
XY25,441250.0,8
SR,265050.0,2
X5K,107797.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3410271.0,25
M-SC,1242820.0,15
X-MC,1081590.0,11
X-LC,1031388.0,13
X-SC,527090.0,6
H-MC,344845.0,3
H-LC,283318.0,3
M-LC,281515.0,4
L-SC,249998.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1174728.0      6
           AR         837658.0      5
           XR         808174.0      7
M-SC       XY24       685325.0      6
X-MC       XY24       587617.0      4
X-LC       X40        474945.0      5
           XY24       306243.0      2
X-SC       X40N       286426.0      4
H-SC       OX40N      267619.0      4
           SR         265050.0      2
X-MC       X40        215658.0      4
H-LC       AR         207719.0      2
X-LC       X40N       184589.0      3
H-MC       XY24       180235.0      1
M-MC       XY24       169372.0      1
X-SC       XY24       163474.0      1
X-MC       XY25       162063.0      2
M-SC       AR         161743.0      2
L-SC       XR         161012.0      2
M-SC       OX40N      143238.0      4
           XY25       135915.0      1
           XR         116599.0      2
X-MC       X40N       116252.0      1
M-LC       XR         110831.0      1
           X5K        107797.0      1
H-MC       XR          92208.0      1
L-SC       OX40N       88986.0      1
X-SC       X40         77190.0      1
H-LC       X200        75599.0      1
H-MC       OX40N       72402.0      1
X-LC       XY25        65611.0      3
L-MC       XR          59395.0      1
H-SC       MH          57042.0      1
L-LC       XY25        39591.0      1
M-LC       XY25        38070.0      1
           XY24        24817.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
